In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import skimage.io as io
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import os
import pickle
import pydicom
from metrics import Pfbeta

In [4]:
DATA_DIR = Path('input/rsna-breast-cancer-detection/')
train = pd.read_csv(DATA_DIR/'train.csv')

In [5]:
from configs import *
from kuma_utils.utils import sigmoid
from sklearn.metrics import roc_auc_score
from metrics import *

# Simple blend

In [18]:
configs_to_include = [
    Aug07lr0(), Res02mod3(), Res02lr0()
]
config_names = [cfg.name for cfg in configs_to_include]

for icfg, cfg in enumerate(configs_to_include):
    df = pd.read_csv(f'input/rsna-breast-cancer-detection/oofs/kuma_{cfg.name}.csv')
    if icfg == 0:
        oof_df = df.rename({'pred': cfg.name}, axis=1)
    else:
        oof_df = oof_df.merge(df.rename({'pred': cfg.name}, axis=1), on=['prediction_id', 'site_id', 'fold', 'target'], how='left')

oof_df

,fold,aug_07_lr0,prediction_id,site_id,target,res_02_mod3,res_02_lr0
0,0,0.002173,49_L,1,0.0,0.001636,0.002996
1,0,0.001864,49_R,1,0.0,0.000749,0.001305
2,0,0.002714,72_L,1,0.0,0.000701,0.000846
3,0,0.002600,72_R,1,0.0,0.001739,0.002242
4,0,0.001508,105_L,2,0.0,0.008912,0.034717
...,...,...,...,...,...,...,...
23821,3,0.001559,65477_R,2,0.0,0.003317,0.001580
23822,3,0.001712,65495_L,1,0.0,0.001959,0.004051
23823,3,0.001139,65495_R,1,0.0,0.001113,0.001197
23824,3,0.002427,65530_L,1,0.0,0.002463,0.003995


In [19]:
def get_score(approx, target):
    metric = PercentilePfbeta(percentile_range=(97.5, 99.0), n_trials=30)
    score, percentile, threshold = metric(approx, target)
    return score, percentile, threshold

In [20]:
scores = []
for fold in range(4):
    oof_fold = oof_df.query('fold == @fold')
    blend_approx = oof_fold[config_names].mean(1).values
    score, percentile, threshold = get_score(blend_approx, oof_fold['target'].values)
    print(f'{fold}: {score:.5f} ({threshold:.5f})')
    scores.append(score)
print(f'Mean: {np.mean(scores):.5f}')


0: 0.55462 (0.41671)
1: 0.54717 (0.39315)
2: 0.51397 (0.43118)
3: 0.53942 (0.30551)
Mean: 0.53879


# Rejection ensemble

In [81]:
def predict_test(cfg, test, fold): # dummy
    metric = PercentilePfbeta(percentile_range=(97.5, 99.0), n_trials=30)
    oof = pd.read_csv(f'input/rsna-breast-cancer-detection/oofs/kuma_{cfg.name}.csv').query('fold == @fold')
    res = test.merge(
        oof[['prediction_id', 'site_id', 'pred']], 
        on='prediction_id', how='left')
    thresholds = []
    for site in [1, 2]:
        oof_site = oof.query('site_id == @site')
        score, _, thres = metric(oof_site['pred'].values, oof_site['target'].values)
        thresholds.append(thres)
    return res.rename({'pred': cfg.name}, axis=1), thresholds


def inference_stage(configs, test_df, fold):
    thresholds_all = []
    model_names = []
    for icfg, cfg in enumerate(configs):
        results, thresholds = predict_test(cfg, test_df, fold)
        # print(cfg.name, thresholds)
        thresholds_all.append(thresholds)
        model_names.append(cfg.name)
        if icfg == 0:
            results_all = results
        else:
            results_all = results_all.merge(results.drop(['site_id', 'target'], axis=1), on='prediction_id', how='left')
    return results_all.drop('target', axis=1), thresholds_all


def weighted_average(df, weights):
    assert df.shape[1] == len(weights)
    res = np.zeros(df.shape[0])
    for i in range(len(weights)):
        res += df.iloc[:, i].values * weights[i] / sum(weights)
    return res

In [105]:
ensemble_configs = [
    {
        'configs': [Res02mod3(), Aug07lr0()],
        'global_weights': [1, 1],
        'drop_rate': 0.50,
        'criteria': 'low value',
    },
    {
        'configs': [Res02lr0(), Aug07lr0pl0()],
        'global_weights': [1, 1],
        'drop_rate': 0.5,
        'criteria': 'low value',
    },
    {
        'configs': [Res02pl1(), Res02mod3pl0(), Res02mod2()],
        'global_weights': [1, 1, 1],
        'drop_rate': None,
        'criteria': 'low value',
    },
]

THRESHOLD_MODE = 'OOF_MEAN_PER_SITE'
assert THRESHOLD_MODE in [
    'OOF', 'OOF_PER_SITE', 'OOF_MEAN_PER_SITE',
    'PROPORTIONAL', 'PROPORTIONAL_PER_SITE', 
    'PROPORTIONAL_MEAN', 'PROPORTIONAL_MEAN_PER_SITE']

metric = PercentilePfbeta(percentile_range=(97.5, 99.0), n_trials=30)


train = pd.read_csv('train_with_fold.csv')
train['prediction_id'] = train['patient_id'].astype(str) + '_' + train['laterality']
train = train.groupby('prediction_id').agg({'cancer': 'max', 'fold': 'max'}).reset_index()
train.head(), train.shape

(  prediction_id  cancer  fold
 0       10006_L       0     0
 1       10006_R       0     0
 2       10011_L       0     3
 3       10011_R       0     3
 4       10025_L       0     2,
 (23826, 3))

In [106]:
all_scores = []
for fold in range(4):
   test = train.query('fold == @fold').drop('fold', axis=1).rename({'cancer': 'target'}, axis=1)
   test2 = test.copy()
   weights_all = []
   models_all = []
   thresholds_all = []
   final_results = []
   for istage, stage_cfg in enumerate(ensemble_configs):
      print(f'[STAGE {istage}: {len(test2)} records] ')
      models_all += [cfg.name for cfg in stage_cfg['configs']]
      weights_all += stage_cfg['global_weights']
      
      results_stage, thresholds_stage = inference_stage(stage_cfg['configs'], test2, fold)
      thresholds_all += thresholds_stage
      if istage == 0:
         results_all = results_stage
      else:
         results_all = results_all.merge(results_stage, on=['prediction_id', 'site_id'], how='inner')
      print(istage, results_all.shape)
      results_all['cancer'] = weighted_average(
         results_all[models_all], weights_all)
      if stage_cfg['criteria'] == 'low value':
         if stage_cfg['drop_rate'] is not None:
               drop_results = results_all.loc[
                  results_all['cancer'] <= np.percentile(results_all['cancer'], stage_cfg['drop_rate']*100)].copy()
               drop_results['cancer'] = 0.
               final_results.append(drop_results)
               test2 = test2.loc[~test2['prediction_id'].isin(drop_results['prediction_id'])]
         else: # final stage
               thresholds = np.average(np.array(thresholds_all), axis=0, weights=weights_all)
               print(thresholds)
               results_all.loc[results_all['site_id'] == 1, 'cancer'] = \
                  (results_all.loc[results_all['site_id'] == 1, 'cancer'] > thresholds[0]).astype(float)
               results_all.loc[results_all['site_id'] == 2, 'cancer'] = \
                  (results_all.loc[results_all['site_id'] == 2, 'cancer'] > thresholds[1]).astype(float)
               final_results.append(results_all)
      else:
         raise NotImplementedError()
   final_results = pd.concat(final_results, axis=0)
   test = test.merge(final_results[['prediction_id', 'cancer']], on='prediction_id', how='left')
   score = metric.pfbeta(test['target'].values, test['cancer'].values)
   print(f'FOLD {fold}: {score:.5f}')
   all_scores.append(score)

[STAGE 0: 5956 records] 
0 (5956, 4)
[STAGE 1: 2978 records] 
1 (2978, 7)
[STAGE 2: 1489 records] 
2 (1489, 10)
[0.57353503 0.49449142]
FOLD 0: 0.53636
[STAGE 0: 5958 records] 
0 (5958, 4)
[STAGE 1: 2979 records] 
1 (2979, 7)
[STAGE 2: 1489 records] 
2 (1489, 10)
[0.5049346  0.49229196]
FOLD 1: 0.54922
[STAGE 0: 5956 records] 
0 (5956, 4)
[STAGE 1: 2978 records] 
1 (2978, 7)
[STAGE 2: 1489 records] 
2 (1489, 10)
[0.4848704  0.40665034]
FOLD 2: 0.52381
[STAGE 0: 5956 records] 
0 (5956, 4)
[STAGE 1: 2978 records] 
1 (2978, 7)
[STAGE 2: 1489 records] 
2 (1489, 10)
[0.46473561 0.43976252]
FOLD 3: 0.50943


In [107]:
all_scores

[0.5363636363636363,
 0.5492227979274611,
 0.5238095238095237,
 0.5094339622641509]

In [108]:
np.mean(all_scores)

0.5297074800911931